This file contains the steps needed to train, evaluate and test the classifier. The current approach is image-level classification.
*Later, we can use the features vector to mask the fire pixels in the fire-image.*

In [ ]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from skimage.feature import local_binary_pattern

## 1. Model Definition

In [ ]:
# Load dataset
fire_dir = "dataset/fire"
non_fire_dir = "dataset/non_fire"

X, y = [], [] #X is the features and y is the Label

# Process fire images, extract_features to select predefine features based on color spaces or textures and assign label
for file in os.listdir(fire_dir):
    img = cv2.imread(os.path.join(fire_dir, file))
    if img is not None:
        features = extract_features(img)
        X.append(features)
        y.append(1)  # Label: Fire:1

# Process non-fire images
for file in os.listdir(non_fire_dir):
    img = cv2.imread(os.path.join(non_fire_dir, file))
    if img is not None:
        features = extract_features(img)
        X.append(features)
        y.append(0)  # Label: Non-Fire:0


X = np.array(X)
y = np.array(y)

# Split into Train & Test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Training samples: {len(X_train)}, Testing samples: {len(X_test)}")


## 2. Model Training and Hyper Parameter Tuning

Train SVM Classifier with Hyperparameter Tuning. We use GridSearchCV to find the best hyperparameters 

Assumption is that features have been extracted and added to X.

In [ ]:
# Define SVM model with RBF kernel
svm = SVC(kernel='rbf', class_weight='balanced')# we might change the class_weight to imbalance after investigating the dataset. in that case, SMOTE or other techniques are used.

# Hyperparameter tuning
param_grid = {
    'C': [0.1, 1, 10, 100],  # Regularization parameter
    'gamma': [0.01, 0.1, 1, 10]  # Kernel coefficient
}

grid_search = GridSearchCV(svm, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Best parameters
print(f"Best Hyperparameters: {grid_search.best_params_}")

# Train final model with best parameters
best_svm = grid_search.best_estimator_


## 3. Model Evaluation

In [ ]:
# Predictions
y_pred = best_svm.predict(X_test)

# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy:.4f}")

# Classification Report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:")
print(conf_matrix)

# Plot Confusion Matrix
plt.figure(figsize=(5, 4))
plt.imshow(conf_matrix, cmap="Blues", interpolation="nearest")
plt.colorbar()
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.xticks([0, 1], ["Non-Fire", "Fire"])
plt.yticks([0, 1], ["Non-Fire", "Fire"])
plt.show()


## 4. Stats on New Unseen Image (tests)

In [ ]:
def classify_image(image_path, model):
    img = cv2.imread(image_path)
    features = extract_features(img).reshape(1, -1)  # Reshape for model
    prediction = model.predict(features)[0]
    
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.title("Fire Detected" if prediction == 1 else "No Fire Detected")
    plt.show()
    
    return prediction

# Test on a new image
image_path = "dataset/test/fire_example.jpg"
result = classify_image(image_path, best_svm)
